# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from sqlite database
engine = create_engine('sqlite:////home/workspace/disaster_response.db')
df = pd.read_sql_table('disaster_response_table', con=engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [24]:
df.head(1)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.message.values

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    input: 
        raw texts and returns
    output: 
        clean_tokens- cleaned list of words in the text after tokenizing and lemmatizing
    '''
    
    text = re.sub(r'[^A-Za-z0-9]'," ",text.lower()) #NORMALIZE- make lowercase, remove character not alphanumeric

    tokens = word_tokenize(text) #TOKENIZE - returns a list of words(toks) in the input text
    
    lemmatizer = WordNetLemmatizer() #initialize lemmatizer
    
    #stopwords removal and strip of whitespace
    stop_tokens = [word.strip() for word in tokens if word not in stopwords.words("english")]
    
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in stop_tokens] # LEMMATIZING
    
    return clean_tokens

    
    
    
    
    
    
""""
#stopwords removal and strip of whitespace
stop_tokens = []
for word in tokens:
    if word not in stopwords.words("english"):
        stop_tokens.append(word.strip())

# LEMMATIZING    
clean_tokens = []
for tok in stop_tokens:
    clean_tok = lemmatizer.lemmatize(tok)
    clean_tokens.append(clean_tok)

return clean_tokens
"""


#TO CHECK THE FUNCTION
text ="My? name wouldn't szound but what can i be applying! Hoping this is successful or  what you thinking"
tokenize(text)

['name', 'szound', 'applying', 'hoping', 'successful', 'thinking']

In [6]:

WordNetLemmatizer().lemmatize("hypotheses")

'hypothesis'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer


from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score,precision_recall_curve,accuracy_score


from sklearn.model_selection import GridSearchCV

In [8]:
#FEATURE EXTRACTION-bagofwords and tfidf
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('multi_clf',MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
])
 

In [19]:
# Split the dataset into features and labels sets
# This is a multi-output classificatn that will use the message col to predict classificatns for 36 categories. from the 4th col to 40th col 
X = df.message.values
y = df.iloc[:, 4:].values


print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
#train classifier
pipeline.fit(X_train, y_train) 

#predict on test feature
y_predict = pipeline.predict(X_test)



/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
category_colnames=['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
                   'search_and_rescue', 'security',
                   'military', 'child_alone', 'water', 'food', 
                   'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death',
                   'other_aid', 'infrastructure_related',
                   'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops',
                   'aid_centers', 'other_infrastructure', 
                   'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
                   'other_weather', 'direct_report']

In [32]:
print(classification_report(y_test, y_predict, target_names=category_colnames))

ValueError: Unknown label type: (array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.